In [5]:
import pandas as pd
from tqdm import tqdm_notebook

prefix = 'data/'

In [16]:
#load train data
train_df = pd.read_csv(prefix + 'train.csv', header=None)
#print(train_df.columns.tolist())
cols = train_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
train_df = train_df[cols]
#print(train_df.columns.tolist())
#train_df = train_df[["1","0"]]
train_df.head()

[0, 1]
[1, 0]


,1,0
0,Slide 1 of 23EXPLORING THE RUINS OF VIJAYANAGA...,0
1,Full text is unavailable for this digitized ar...,4
2,The American Government is shouldering a modes...,9
3,To the Editor:Re ''Consulting: Don't Lose It I...,9
4,Full text is unavailable for this digitized ar...,6


In [17]:
#load test data
test_df = pd.read_csv(prefix + 'test.csv', header=None)
cols = test_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
test_df = test_df[cols]
test_df.head()

,1,0
0,Credit... The New York Times ArchivesAugust 1 ...,5
1,Full text is unavailable for this digitized ar...,4
2,Credit... The New York Times ArchivesFebruary ...,3
3,SHAPIRO--Pauline. The Yeshiva University famil...,0
4,Most presidents will at some point benefit fro...,1


In [18]:
from simpletransformers.classification import ClassificationModel
import pandas as pd


# Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns. If the Dataframe has a header, it should contain a 'text' and a 'labels' column. If no header is present, the Dataframe should contain at least two columns, with the first column is the text with type str, and the second column in the label with type int.
train_data = train_df

eval_data = test_df

eval_df = pd.DataFrame(eval_data)




In [22]:
# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased',use_cuda=False, num_labels=10, args={'reprocess_input_data': True, 'overwrite_output_dir': True}) 
# You can set class weights by using the optional weight argument

100%|██████████| 435779157/435779157 [00:33<00:00, 12988933.75B/s]


In [23]:


# Train the model
model.train_model(train_df)

Converting to features started.


AttributeError: 'float' object has no attribute 'split'

In [ ]:


# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:

predictions, raw_outputs = model.predict(["Some arbitary sentence"])